In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
#from static_grader import grader

#this is the grader used by WorldQuant University, it is only avaliable on their server hence, I commented it out

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [ ]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [2]:
import pandas as pd
import numpy as np
import gzip

In [3]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as f:
    scripts = pd.read_csv(f)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [4]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
with gzip.open('./dw-data/practices.csv.gz', 'rb') as f:
    practices = pd.read_csv(f, names=col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [5]:
with gzip.open('./dw-data/chem.csv.gz', 'rb') as f:
    chem = pd.read_csv(f)
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [6]:
stats = scripts.describe()

In [7]:
summary_stats = []
for i in stats:
    total = stats[i]['count'] * stats[i]['mean']
    summary_stats.append((i, tuple([total] + [stats[i][j] for j in ['mean', 'std', '25%', '50%', '75%']])))

In [8]:
print(summary_stats)

[('items', (8888304.0, 9.133135976111625, 29.20419828297713, 1.0, 2.0, 6.0)), ('nic', (71100424.84000827, 73.05891517921756, 188.07025690683025, 7.8, 22.64, 65.0)), ('act_cost', (66164096.11999956, 67.98661326170611, 174.40170332300627, 7.33, 21.22, 60.67)), ('quantity', (721457006.0, 741.3298348837282, 3665.426958468499, 28.0, 100.0, 350.0))]


In [ ]:
#grader.score.dw__summary_statistics(summary_stats)

#==================
#Your score:  1.0
#==================

## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [9]:
grp_bnf_name = scripts.groupby('bnf_name')['items'].sum()
grp_bnf_name = grp_bnf_name.reset_index()
bnf_with_max_item = grp_bnf_name[grp_bnf_name['items'] == grp_bnf_name.max()['items']].reset_index()

In [10]:
most_common_item = [(bnf_with_max_item['bnf_name'][0], bnf_with_max_item['items'][0])]

In [11]:
print(most_common_item)

[('Omeprazole_Cap E/C 20mg', 218583)]


In [ ]:
#grader.score.dw__most_common_item(most_common_item)

#==================
#Your score:  1.0
#==================

## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [12]:
grp_code = practices.groupby('code').agg({'post_code': 'min', 'name': 'min'}).reset_index().rename(\
                                        columns = {'code': 'practice','name': 'prac_name'})
grp_code.head()

,practice,post_code,prac_name
0,A81001,TS18 1HU,THE DENSHAM SURGERY
1,A81002,TS18 2AW,QUEENS PARK MEDICAL CENTRE
2,A81003,TS25 1QU,VICTORIA MEDICAL PRACTICE
3,A81004,TS1 3BE,BLUEBELL MEDICAL CENTRE
4,A81005,TS14 7DJ,SPRINGWOOD SURGERY


In [13]:
joined = scripts.copy()
merged_post = pd.merge(joined, grp_code, how='left', on='practice')
merged_post.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,post_code,prac_name
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,CH44 5UF,GP OOH VCH
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,CH44 5UF,GP OOH VCH
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,CH44 5UF,GP OOH VCH
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,CH44 5UF,GP OOH VCH
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,CH44 5UF,GP OOH VCH


In [14]:
groupby_post_object = merged_post.groupby('post_code')
sum_items_per_post = groupby_post_object.sum()['items']
def max_item_bnf_name(mini_df):
    mini_df = mini_df.groupby('bnf_name').sum().reset_index()
    return mini_df.sort_values(by=['items', 'bnf_name'], ascending=[False, True]).iloc[0]
    
groupby_post_max_item = groupby_post_object.apply(max_item_bnf_name)
groupby_post_max_item['frac_item'] = groupby_post_max_item['items'] / sum_items_per_post
groupby_post_max_item = groupby_post_max_item.reset_index()

In [15]:
items_by_region = [(groupby_post_max_item['post_code'][i], groupby_post_max_item['bnf_name'][i],\
                    groupby_post_max_item['frac_item'][i]) for i in range(100)]


In [16]:
print(items_by_region)

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.031058906339360346), ('B12 9LP', 'Paracet_Tab 500mg', 0.02489310607391788), ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.027111371172225472), ('B21 9RY', 'Metformin HCl_Tab 500mg', 0.03329358300834757), ('B23 6DJ', 'Lansoprazole_Cap 30mg (E/C Gran)', 0.021384456106529576), ('B61 0AZ', 'Omeprazole_Cap E/C 20mg', 0.028713318284424378), ('B70 7AW', 'Paracet_Tab 500mg', 0.025135992162726547), ('B72 1RL', 'Omeprazole_Cap E/C 20mg', 0.020228765092141495), ('B8 1RZ', 'Metformin HCl_Tab 500mg', 0.021347750961484866), ('B9 5PU', 'Ventolin_Evohaler 100mcg (200 D)', 0.024826024522257815), ('B90 3LX', 'Omeprazole_Cap E/C 20mg', 0.026965103983080718), ('BA5 1XJ', 'Omeprazole_Cap E/C 20mg', 0.028261290947858113), ('BB11 2DL', 'Omeprazole_Cap E/C 20mg', 0.027381741821396993), ('BB2 1AX', 'Omeprazole_Cap E/C 20mg', 0.03428191046763188), ('BB3 1PY', 'Omeprazole_Cap E/C 20mg', 0.032683395995453356), ('BB4 5SL', 'Omeprazole_Cap E/C 20mg', 0.0374

In [ ]:
#grader.score.dw__items_by_region(items_by_region)

#==================
#Your score:  1.0
#==================

## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [17]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [18]:
copy_of_scripts = scripts.copy()
copy_of_chem = chem.copy()
copy_of_chem['is_opioid'] = [int(i) for i in copy_of_chem['NAME'].str.contains('|'.join(opioids), case=False)]
copy_of_chem = copy_of_chem.groupby('CHEM SUB').first()['is_opioid'].reset_index().rename(columns={'CHEM SUB': 'bnf_code'})
copy_of_scripts = pd.merge(copy_of_scripts, copy_of_chem, on='bnf_code', how='left')
copy_of_scripts['is_opioid'].fillna(0, inplace=True)
copy_of_scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,is_opioid
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0.0
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0.0
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0.0
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0.0
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0.0


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [19]:
copy_of_scripts.groupby('practice').count().head()

,bnf_code,bnf_name,items,nic,act_cost,quantity,is_opioid
practice,,,,,,,
A81005,1507,1507,1507,1507,1507,1507,1507
A81007,1454,1454,1454,1454,1454,1454,1454
A81011,1568,1568,1568,1568,1568,1568,1568
A81012,1332,1332,1332,1332,1332,1332,1332
A81017,2150,2150,2150,2150,2150,2150,2150


In [20]:
n_opioids_per_practice = copy_of_scripts.groupby('practice').count()['is_opioid'].reset_index()
opioids_per_practice = copy_of_scripts.groupby('practice').mean()['is_opioid'].reset_index()
opioids_per_practice['n_of_script'] = n_opioids_per_practice['is_opioid']
opioids_per_practice = opioids_per_practice.rename(columns={'is_opioid': 'opioid_rate'})
opioids_per_practice.head()

,practice,opioid_rate,n_of_script
0,A81005,0.033179,1507
1,A81007,0.043329,1454
2,A81011,0.046556,1568
3,A81012,0.042793,1332
4,A81017,0.038140,2150


In [21]:
overall_opioid_rate = copy_of_scripts['is_opioid'].mean()
relative_opioids_per_practice = opioids_per_practice.copy()
relative_opioids_per_practice['d(opioid_rate)'] = opioids_per_practice['opioid_rate'] - overall_opioid_rate 
relative_opioids_per_practice.head()

,practice,opioid_rate,n_of_script,d(opioid_rate)
0,A81005,0.033179,1507,-0.002624
1,A81007,0.043329,1454,0.007526
2,A81011,0.046556,1568,0.010753
3,A81012,0.042793,1332,0.006990
4,A81017,0.038140,2150,0.002337


How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [22]:
from math import sqrt

overall_sd = copy_of_scripts['is_opioid'].std()
standard_error_per_practice_series = overall_sd / (relative_opioids_per_practice['n_of_script'].apply(lambda x: sqrt(x)))
standard_error_per_practice = relative_opioids_per_practice.copy()
standard_error_per_practice['S.E.'] = standard_error_per_practice_series.reset_index().rename(\
                                            columns={'n_of_script': 'S.E.'})['S.E.']
opioid_scores = standard_error_per_practice.copy()
opioid_scores['oscore'] = opioid_scores['d(opioid_rate)'] / opioid_scores['S.E.']
opioid_scores.head()

,practice,opioid_rate,n_of_script,d(opioid_rate),S.E.,oscore
0,A81005,0.033179,1507,-0.002624,0.004786,-0.548306
1,A81007,0.043329,1454,0.007526,0.004873,1.544557
2,A81011,0.046556,1568,0.010753,0.004692,2.291795
3,A81012,0.042793,1332,0.006990,0.005091,1.373060
4,A81017,0.038140,2150,0.002337,0.004007,0.583168


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [23]:
results = pd.merge(opioid_scores, grp_code, on='practice', how='left')
results = results.sort_values(by='oscore', ascending=False).reset_index().drop(columns='index')

anomalies = [(results['prac_name'][i], results['oscore'][i], results['n_of_script'][i]) for i in range(100)]

In [24]:
results.head()

,practice,opioid_rate,n_of_script,d(opioid_rate),S.E.,oscore,post_code,prac_name
0,Y01852,0.857143,7,0.821340,0.070225,11.695818,OL11 1DN,NATIONAL ENHANCED SERVICE
1,Y03006,1.000000,2,0.964197,0.131379,7.339043,M26 2SP,OUTREACH SERVICE NH / RH
2,Y03668,0.183333,60,0.147531,0.023986,6.150582,BS4 1WH,BRISDOC HEALTHCARE SERVICES OOH
3,G81703,0.194444,36,0.158642,0.030966,5.123032,TN34 1BA,H&R P C SPECIAL SCHEME
4,Y04997,0.087227,321,0.051425,0.010370,4.958866,BL9 0NJ,HMR BARDOC OOH


In [25]:
print(anomalies)

[('NATIONAL ENHANCED SERVICE', 11.69581786285154, 7), ('OUTREACH SERVICE NH / RH', 7.339043019185809, 2), ('BRISDOC HEALTHCARE SERVICES OOH', 6.150581749039399, 60), ('H&R P C SPECIAL SCHEME', 5.1230324139960715, 36), ('HMR BARDOC OOH', 4.958866438451784, 321), ('INTEGRATED CARE 24 LTD (CWSX OOH)', 4.888878160447508, 426), ('DARWEN HEALTHCARE', 4.839158968601383, 1917), ('THE LIMES MEDICAL PRACTICE', 4.54684187230158, 1321), ('IC24 LTD (BRIGHTON & HOVE OOH)', 4.335047010573882, 357), ('OLDHAM 7 DAY ACCESS HUB2 OOH', 4.311784036579043, 56), ('IC24 LTD (NORFOLK & WISBECH OOH)', 4.2575005924420095, 489), ('ROSSENDALE MIU & OOH', 4.256827446291742, 18), ('BURY WALK-IN CENTRE', 4.150589122851554, 138), ('IC24 LTD (HORSHAM & MID SUSSEX OOH)', 3.781620703817035, 215), ('LCW HOUNSLOW CCG OOH', 3.5828485461803874, 69), ('WEEKEND WORKING EASINGTON NORTH', 3.565938771104005, 278), ('COMPASS ENFIELD', 3.558720265081086, 7), ('BASSETLAW DRUG & ALCOHOL SERVICE', 3.533264102486019, 2), ('THE PARK SUR

In [ ]:
#grader.score.dw__script_anomalies(anomalies)

#==================
#Your score:  1.0
#==================

## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [26]:
# load the 2016 data
with gzip.open('./dw-data/201606scripts_sample.csv.gz', 'rb') as f:
    scripts16 = pd.read_csv(f)
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [27]:
num_of_preps16_df = scripts16.groupby('bnf_name').count()[['practice']].rename(columns={'practice': 'num_of_preps16'})
num_of_preps17_df = scripts.groupby('bnf_name').count()[['practice']].rename(columns={'practice': 'num_of_preps17'})
growth_df = pd.merge(num_of_preps16_df, num_of_preps17_df, on='bnf_name')
growth_df['growth'] = growth_df['num_of_preps17'] - growth_df['num_of_preps16']
growth_df['growth_rate'] = growth_df['growth'] / growth_df['num_of_preps16']
growth50_df = growth_df[growth_df['num_of_preps16'] >= 50]
growth50_df = growth50_df.sort_values(by='growth_rate', ascending=False).reset_index()
growth50_df.head()

,bnf_name,num_of_preps16,num_of_preps17,growth,growth_rate
0,Butec_Transdermal Patch 5mcg/hr,62,277,215,3.467742
1,Butec_Transdermal Patch 10mcg/hr,69,276,207,3.000000
2,Fostair NEXThaler_Inh 200mcg/6mcg (120D),86,209,123,1.430233
3,Pneumococcal_Vac 0.5ml Vl (23 Valent),193,438,245,1.269430
4,Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,52,118,66,1.269231


In [28]:
largest_growth50 = growth50_df.head(50)
largest_shrinkage50 = growth50_df.tail(50)
script_growth100 = pd.concat([largest_growth50, largest_shrinkage50], ignore_index=True)

In [29]:
script_growth = [(script_growth100['bnf_name'][i], script_growth100['growth_rate'][i], script_growth100['num_of_preps16'][i])\
                 for i in range(100)]

In [30]:
print(script_growth)

[('Butec_Transdermal Patch 5mcg/hr', 3.467741935483871, 62), ('Butec_Transdermal Patch 10mcg/hr', 3.0, 69), ('Fostair NEXThaler_Inh 200mcg/6mcg (120D)', 1.430232558139535, 86), ('Pneumococcal_Vac 0.5ml Vl (23 Valent)', 1.2694300518134716, 193), ('Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev', 1.2692307692307692, 52), ('Trulicity_Inj 1.5mg/0.5ml Pf Pen', 1.1851851851851851, 54), ('CosmoCol_Paed Oral Pdr Sach 6.9g', 1.1774193548387097, 62), ('Dulaglutide_Inj 1.5mg/0.5ml Pf Dev', 0.9634146341463414, 82), ('ViATIM_Vac D/Chamber 160u/25mcg 1ml Pfs', 0.912, 125), ('Empagliflozin_Tab 25mg', 0.896, 125), ('CareSens Lancets 0.31mm/30 Gauge', 0.8955223880597015, 67), ('Fostair_Inh 200mcg/6mcg (120D) CFF', 0.8616600790513834, 253), ('Orbis Nor Saline Sod Chlor 0.9% Nsl Dps', 0.8588235294117647, 85), ('Umeclidinium Brom_Inh 65mcg (30D)', 0.8275862068965517, 87), ('Ultibro Breezhaler_Pdr Inh Cap + Dev', 0.8016528925619835, 121), ('Medihoney_Barrier Crm', 0.7840909090909091, 88), ('Fluoxetine HCl_Oral S

In [ ]:
#grader.score.dw__script_growth(script_growth)

#==================
#Your score:  1.0
#==================

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [31]:
p = 1 / len(copy_of_scripts['bnf_code'].unique())
grp_by_bnf_code = scripts.groupby('bnf_code').count()[['practice']]
grp_by_bnf_code.rename(columns={'practice': 'n_of_preps'}, inplace=True)
#grp_bnf_name['n_of_preps'] = scripts.groupby('bnf_code').count()['practice'].to_list()
grp_by_bnf_code['prep_rate'] = grp_by_bnf_code['n_of_preps'] / grp_by_bnf_code['n_of_preps'].sum()
rare_codes = grp_by_bnf_code[grp_by_bnf_code['prep_rate'] < 0.1*p].reset_index()
#rates = ...
rare_codes = rare_codes['bnf_code'].to_list()

copy_of_scripts['is_rare'] = [i for i in copy_of_scripts['bnf_code'].isin(rare_codes)]
copy_of_scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,is_opioid,is_rare
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0.0,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0.0,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0.0,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0.0,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0.0,False


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [32]:
def cost_from_rare(mini_df):
    act_cost_sum = mini_df.sum()['act_cost']
    mini_df = mini_df[mini_df['is_rare'] == True]
    mini_df = pd.DataFrame({'rare_cost_sum': [mini_df.sum()['act_cost']], 'act_cost': [act_cost_sum]})
    return mini_df.iloc[0]

In [33]:
practice_prep_of_rare_df = copy_of_scripts.groupby('practice').apply(cost_from_rare)
practice_prep_of_rare_df.head()

,rare_cost_sum,act_cost
practice,,
A81005,1247.83,103840.82
A81007,951.06,113482.49
A81011,816.02,159507.03
A81012,1145.11,83296.81
A81017,1712.15,232656.17


In [34]:
practice_prep_of_rare_df['rare_cost_prop'] = practice_prep_of_rare_df['rare_cost_sum'] / practice_prep_of_rare_df['act_cost']
practice_prep_of_rare_df.head()

,rare_cost_sum,act_cost,rare_cost_prop
practice,,,
A81005,1247.83,103840.82,0.012017
A81007,951.06,113482.49,0.008381
A81011,816.02,159507.03,0.005116
A81012,1145.11,83296.81,0.013747
A81017,1712.15,232656.17,0.007359


Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [35]:
scripts_rare_cost_prop = copy_of_scripts[copy_of_scripts['is_rare'] == True]['act_cost'].sum()\
                                                    / copy_of_scripts['act_cost'].sum()
practice_prep_of_rare_df['relative_rare_cost_prop'] = practice_prep_of_rare_df['rare_cost_prop'] - scripts_rare_cost_prop
practice_prep_of_rare_df.head()

,rare_cost_sum,act_cost,rare_cost_prop,relative_rare_cost_prop
practice,,,,
A81005,1247.83,103840.82,0.012017,-0.003946
A81007,951.06,113482.49,0.008381,-0.007582
A81011,816.02,159507.03,0.005116,-0.010847
A81012,1145.11,83296.81,0.013747,-0.002216
A81017,1712.15,232656.17,0.007359,-0.008604


Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [36]:
standard_errors = practice_prep_of_rare_df['relative_rare_cost_prop'].std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [37]:
practice_prep_of_rare_df['rare_scores'] = practice_prep_of_rare_df['relative_rare_cost_prop'] / standard_errors

In [38]:
rare_scripts_df = pd.merge(grp_code, practice_prep_of_rare_df, on='practice').sort_values(\
                            by='rare_scores', ascending=False).reset_index()
rare_scripts_df.head()

,index,practice,post_code,prac_name,rare_cost_sum,act_cost,rare_cost_prop,relative_rare_cost_prop,rare_scores
0,765,Y03472,PE1 2QP,CONSULTANT DIABETES TEAM,67.04,67.04,1.000000,0.984037,16.262687
1,831,Y05320,B61 0AZ,DMC COMMUNITY DERMATOLOGY RBWF,1753.92,1883.14,0.931381,0.915418,15.128648
2,793,Y04404,SM6 0HY,OUTPATIENTS JUBILEE HEALTH CENTRE,702.93,1488.22,0.472329,0.456366,7.542139
3,766,Y03484,ME12 3LT,DMC COMMUNITY DERMATOLOGY CLINIC,973.12,2129.81,0.456905,0.440942,7.287222
4,794,Y04424,SS8 0JA,DMC HEALTHCARE,91.44,212.77,0.429760,0.413797,6.838614


In [39]:
rare_scripts = [(rare_scripts_df['practice'][i], rare_scripts_df['prac_name'][i], rare_scripts_df['rare_scores'][i])\
                for i in range(100)]

In [40]:
print(rare_scripts)

[('Y03472', 'CONSULTANT DIABETES TEAM', 16.262687124655088), ('Y05320', 'DMC COMMUNITY DERMATOLOGY RBWF', 15.128648195416881), ('Y04404', 'OUTPATIENTS JUBILEE HEALTH CENTRE', 7.542139356104626), ('Y03484', 'DMC COMMUNITY DERMATOLOGY CLINIC', 7.287222200297833), ('Y04424', 'DMC HEALTHCARE', 6.838614181432871), ('Y00631', 'BINGLEY DERMATOLOGY CLINIC', 5.755997549837744), ('A89038', 'BARMSTON MEDICAL CENTRE', 5.6649403685399395), ('Y01696', 'BASSETLAW HOSPICE OF THE GOOD SHEPHERD', 4.290685171474925), ('Y03699', 'OLDHAM DERMATOLOGY SERVICE', 4.103743999157247), ('Y02045', 'VERNOVA HEALTHCARE CIC', 3.180715470414401), ('Y02823', 'DMC VICARAGE LANE', 3.054121649498704), ('Y00997', 'COMMUNITY DERMATOLOGY SERVICE', 3.0181283595352792), ('Y05019', 'OLDHAM TOTAL SKIN SERVICE', 2.7247515958989306), ('Y01003', 'BURY OOH', 2.3504363471398597), ('Y05431', 'NOTTS APPLIANCE MANAGEMENT SERVICE', 2.037158543233388), ('Y03557', 'IPU 10-17 RECOVERY COVENTRY', 2.0304495001187806), ('Y05419', 'WEEKEND WORK

In [ ]:
#grader.score.dw__rare_scripts(rare_scripts)

#==================
#Your score:  1.0
#==================